# WandB results

In [ ]:
import wandb
import numpy as np
import torch
import pandas as pd

In [ ]:
def flatten_log(log):
    out = {}
    for k, v in log.items():
        if isinstance(v, dict):
            for k2, v2 in v.items():
                out[f"{k}.{k2}"] = v2
        elif isinstance(v, list):
            if len(v) == 1:
                out[k] = v[0]
        else:
            out[k] = v
    return out


def handle_v(v):
    if isinstance(v, list):
        return handle_v(v[0])
    if isinstance(v, np.float32):
        return v.item()
    if isinstance(v, torch.Tensor):
        # Handle torch tensors
        if v.numel() == 1:
            return v.item()
        else:
            # Return the numpy array if it has more than one element
            return v.cpu().numpy()
    return v

In [ ]:
def get_wandb_logs(api, run_path):
    # Fetch the run
    run = api.run(run_path)
    scan_history = run.scan_history()  # Fetch the scan history

    # Flatten the logs
    flattened_logs = [flatten_log(log) for log in scan_history]
    all_data = pd.DataFrame(flattened_logs)
    return all_data

In [ ]:
def compute_deltas(data, up_to_step=15, max_or_min="max"):
    data = data[
        [
            "step",
            "pop_best_eval",
            "median_eval",
            "mean_eval",
            "prompt",
            "running_time",
            "memory",
            "best_img.path"
        ]
    ]

    step_0 = data[data["step"] == 0]
    all_other_steps = data[data["step"] != 0]

    baseline_pop_best = step_0.groupby("prompt")["pop_best_eval"].agg(["mean"])
    pop_bests = (
        all_other_steps.loc[all_other_steps["step"] <= up_to_step]
        .groupby("prompt")[["pop_best_eval", "mean_eval", "median_eval"]]
        .agg([max_or_min])
    )

    pop_bests.columns = pop_bests.columns.map("_".join)
    joined_data = baseline_pop_best.join(pop_bests)
    joined_data["delta_max"] = joined_data[f"pop_best_eval_{max_or_min}"] - joined_data["mean"]
    joined_data["delta_mean"] = joined_data[f"mean_eval_{max_or_min}"] - joined_data["mean"]
    joined_data["delta_median"] = joined_data[f"median_eval_{max_or_min}"] - joined_data["mean"]

    # joined_data = joined_data.rename(columns={"mean": "baseline", "pop_best_eval_max": "pop_best"})
    joined_data = joined_data.reset_index()
    joined_data = joined_data.rename(
        columns={
            "mean": "baseline",
            f"pop_best_eval_{max_or_min}": "pop_best",
            f"mean_eval_{max_or_min}": "mean",
            f"median_eval_{max_or_min}": "median",
            "delta_max": "improvement_max",
            "delta_mean": "improvement_mean",
            "delta_median": "improvement_median",
        }
    )

    # Get the best image paths
    subset = all_other_steps.loc[all_other_steps["step"] <= up_to_step]
    best_img_indices = subset.groupby("prompt")[["pop_best_eval"]].idxmax()
    best_rows = subset.loc[best_img_indices.values.flatten()]
    best_img_paths = best_rows[["prompt", 'best_img.path']].set_index("prompt")
    joined_data = joined_data.join(best_img_paths, on="prompt")
    return joined_data

In [ ]:
def compute_deltas_dnos(data, up_to_step=15, max_or_min="max"):
    data = data[["Step", "Prompt", "Reward"]]

    step_0 = data[data["Step"] == 0]
    all_other_steps = data[data["Step"] != 0]

    baseline_pop_best = step_0.groupby("Prompt")["Reward"].agg(["mean"])
    pop_bests = (
        all_other_steps.loc[all_other_steps["Step"] <= up_to_step]
        .groupby("Prompt")["Reward"]
        .agg([max_or_min])
    )
    joined_data = pop_bests.join(baseline_pop_best)
    joined_data["delta"] = joined_data[max_or_min] - joined_data["mean"]
    joined_data = joined_data.rename(columns={"mean": "baseline", f"{max_or_min}": "pop_best"})
    joined_data = joined_data.reset_index()
    joined_data = joined_data.rename(columns={"mean": "baseline", f"{max_or_min}": "pop_best", "delta": "improvement", "Prompt": "prompt"})
    return joined_data

In [ ]:
def add_fields(data, fields):
    for col, val in fields.items():
        data[col] = val
    return data

In [ ]:
# Initialize WandB
api = wandb.Api(timeout=60)

## SD Random

### Img Reward

In [157]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/skl06y3a")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,-0.031075,1.406665,0.273785,0.382568,1.437740,0.304860,0.413643
std,1.082906,0.519118,0.752042,0.843772,0.962182,0.724381,0.734997
min,-2.268306,-0.568638,-1.496632,-1.717174,-0.028912,-2.110340,-1.954732
25%,-0.714113,1.189102,-0.307636,-0.188857,0.725083,-0.152275,-0.069230
50%,0.043865,1.582806,0.311675,0.437331,1.286778,0.172543,0.279640
75%,0.769696,1.783020,0.882153,0.995918,1.966600,0.775642,0.831820
max,1.906958,1.988246,1.784922,1.877474,3.987686,2.920750,3.081417


### HPSv2

In [ ]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/bzquoitz")
compute_deltas(data, up_to_step=15).describe()

### CLIP

In [ ]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/bfzpae40")
compute_deltas(data, up_to_step=15).describe()

### JPEG

In [ ]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/miy93gst")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

## SD Zero-order Img Reward

### Img Reward

In [158]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/4ghfz8nm")
compute_deltas(data, up_to_step=15).describe()

,baseline,pop_best,mean,median,improvement_max,improvement_mean,improvement_median
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.035915,1.454864,0.820133,0.965780,1.418948,0.784218,0.929865
std,1.058178,0.529670,0.704283,0.733212,0.990894,0.844804,0.945048
min,-2.229451,-0.407780,-1.401736,-1.531963,0.078075,-0.672914,-0.474530
25%,-0.672078,1.237632,0.330920,0.453183,0.625569,0.138108,0.226298
50%,0.137491,1.673038,0.893824,1.093121,1.203078,0.568623,0.668256
75%,0.847085,1.851148,1.400938,1.563997,2.002062,1.249367,1.396598
max,1.876520,2.001970,1.966297,1.976373,3.984655,3.532083,3.745600


### HPSv2

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/rwijixwm")
compute_deltas(data, up_to_step=15).describe()

### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/ullx2qou")
compute_deltas(data, up_to_step=15).describe()

### JPEG

In [ ]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/gz3b6q2k")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

## DNO

### Img Reward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-dno/9vis1gzw")
compute_deltas_dnos(data, up_to_step=15).describe()

### HPSv2

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-dno/h26tfx2i")
compute_deltas_dnos(data, up_to_step=15).describe()

### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-dno/c3otmlnd")
compute_deltas_dnos(data, up_to_step=15).describe()

### JPEG

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-dno/6enuouop")
compute_deltas_dnos(data, up_to_step=15, max_or_min="min").describe()

## SD Cosyne

### Img Reward

In [ ]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/p5selqpd")
compute_deltas(data, up_to_step=15).describe()

### HPSv2

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/4p9ka0yv")
compute_deltas(data, up_to_step=15).describe()

### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/lxdmzzpo")
compute_deltas(data, up_to_step=15).describe()

### JPEG

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/5g3otp1e")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

### Rot

### Image Reward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/sspqm9uy")
compute_deltas(data, up_to_step=15).describe()

### HPSv2

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/ktgd80b8")
compute_deltas(data, up_to_step=15).describe()

### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/9f6jrs07")
compute_deltas(data, up_to_step=15).describe()

### JPEG

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/xcva28u6")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

## SD SNES

### Img Reward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/yxq7l3oo")
compute_deltas(data, up_to_step=15).describe()

### HPSv2

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/l1461i7v")
compute_deltas(data, up_to_step=15).describe()

### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/j1yao8sr")
compute_deltas(data, up_to_step=15).describe()

### JPEG

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/pw8cxs22")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

### Rot

### Image Reward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/91rwif2q")
compute_deltas(data, up_to_step=15).describe()

### HPSv2

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/ll4ozqve")
compute_deltas(data, up_to_step=15).describe()

### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/1fdjdwx6")
compute_deltas(data, up_to_step=15).describe()

### JPEG

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/zmwqycd2")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

## SD PGPE

### Img Reward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/w5sjg6wc")
compute_deltas(data, up_to_step=15).describe()

### HPSv2

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/5c0hh9jn")
compute_deltas(data, up_to_step=15).describe()

### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/jd8c8l67")
compute_deltas(data, up_to_step=15).describe()

### JPEG

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/1wo6mnfn")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

### Rot

### Image Reward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/heiropgw")
compute_deltas(data, up_to_step=15).describe()

### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/f4709ql9")
compute_deltas(data, up_to_step=15).describe()

### HPSv2

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/8el737q9")
compute_deltas(data, up_to_step=15).describe()

### JPEG

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/k3sgr4xu")
compute_deltas(data, up_to_step=15, max_or_min="min").describe()

# Open Image Preferences

## SD3

### Random

#### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/pym27hnm")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

#### ImageReward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/ddwgfeu6")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

### Zero-Order

#### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/cvxomgxz")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

#### ImageReward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/2621vyt4")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

### CoSyne

#### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/wg7e1kib")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

#### ImageReward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/rwouqy0p")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

### SNES

#### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/tp90h3fv")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

#### ImageReward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/iq0vlq8v")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

### PGPE

#### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/lovj86kc")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

#### ImageReward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/3uvnm0gm")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

## PixArt

### Random

#### CLIP

In [ ]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/huuj70g6")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

#### ImageReward

In [ ]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/0q4jjz0r")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

### Zero-Order

#### CLIP

In [ ]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/cxkdtse9")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

#### ImageReward

In [ ]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/ndlxn1i8")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

### CoSyne

#### CLIP

In [ ]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/id3fc40g")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

#### ImageReward

In [ ]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/en5e7do2")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

### SNES

#### CLIP

In [ ]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/4z67tfcf")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

#### ImageReward

In [ ]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/2ip5kgez")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

### PGPE

#### CLIP

In [ ]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/qp92xk2i")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

#### ImageReward

In [ ]:
data = get_wandb_logs(api, "seris/inference-diffusion-noise-optim/fvunpi6b")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

## SD

### Random

#### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/uy53odzi")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

#### ImageReward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/z9hzxb7n")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

### Zero-Order

#### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/a44squpo")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

#### ImageReward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/lguscqhu")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

### CoSyne

#### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/egyrhyxk")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

#### ImageReward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/vloeu4z3")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

### SNES

#### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/7iiytsny")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

#### ImageReward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/pyxzelxn")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

### PGPE

#### CLIP

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/bt0numtw")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

#### ImageReward

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim/s1foby3g")
compute_deltas(data, up_to_step=15, max_or_min="max").describe()

# Steps Ablation

In [ ]:
def get_step_data(data):
    data = data.groupby("step")[
        [
            "pop_best_eval",
            "median_eval",
            "mean_eval",
            # "prompt",
            # "best_img.path",
            # "running_time",
            # "memory",
        ]
    ].agg(["mean", "std", "median", "max", "min"])
    data.columns = data.columns.map("_".join)
    return data

In [ ]:
def get_dno_step_data(data):
    data.rename(columns={"Step": "step", "Prompt": "prompt"}, inplace=True)
    data['pop_best_eval'] = data['Reward']
    data['median_eval'] = data['Reward']
    data['mean_eval'] = data['Reward']
    data = data.groupby("step")[
        [
            "pop_best_eval",
            "median_eval",
            "mean_eval",
        ]
    ].agg(["mean", "std", "median", "max", "min"])
    data.columns = data.columns.map("_".join)
    return data

## SD

### Random

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim-ablation/tos3uoc9")

In [ ]:
random_step_data = get_step_data(data).reset_index()
random_step_data['algorithm'] = "Random"

### Zero-Order

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim-ablation/mdunyl2u")

In [ ]:
zero_order_step_data = get_step_data(data).reset_index()
zero_order_step_data['algorithm'] = "Zero Order"

### Cosyne

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim-ablation/jbpl4c7s")

In [ ]:
cosyne_step_data = get_step_data(data).reset_index()
cosyne_step_data['algorithm'] = "Cosyne"

### SNES

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim-ablation/vsv2jra3")

In [ ]:
snes_step_data = get_step_data(data).reset_index()
snes_step_data['algorithm'] = "SNES"

### PGPE

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-diffusion-noise-optim-ablation/zan3ho1t")

In [ ]:
pgpe_step_data = get_step_data(data).reset_index()
pgpe_step_data['algorithm'] = "PGPE"

In [ ]:
pd.concat([
    random_step_data,
    zero_order_step_data,
    cosyne_step_data,
    snes_step_data,
    pgpe_step_data
]).to_csv("sd_open_img_60_for_steps_ablation1.csv", index=False)

In [ ]:
data = get_wandb_logs(api, "pjajal/inference-dno/dcil1pw1")
get_dno_step_data(data)